Alexander S. Lundervold, November 1st, 2019

<img width=30% src="../assets/uc.png">

# Setup

In [1]:
%matplotlib inline
from pathlib import Path 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
# To make the notebook reproducible
seed = 42
np.random.seed(seed)

# Introduction

As you know very well by now, machine learning models typically have a number of _hyperparameters_ that has to be chosen correctly to get higher performance.

> **What is a hyperparameter?** During training the model's _parameters_ are automatically tuned to make the model produce useful outputs. This is typically achieved by using an optimization algorithm (for example gradient descent) to minimize some cost function (for example mean square error or cross-entropy). 

> However, there are typically other parameters in the model that are not automatically tuned during training. These are the things you pass to the scikit-learn estimators as parameters. For example `RandomForestClassifier(max_depth = 2, n_estimators = 100, ...)` They are called _hyperparameters_. Examples include things like the learning rate, the amount of regularization, the number of layers in a neural network (as we shall see), and much more. Some models have a large number of such parameters which can influence their performance heavily. 

**How do we select good hyperparameters?**

It's essentially a learning task: train the model to also obtain good hyperparameter settings. However, it's typically not that easy to formulate the task in a way where machine learning training methods can work (it's for example difficult to create cost functions for this task that can be optimized using gradient descent, since such cost functions wouldn't be differentiable). 

> **There are some very interesting methods to make powerful models that optimize ML models, but that is beyond our scope in this notebook**. Have a look at <a href="https://ai.googleblog.com/2017/05/using-machine-learning-to-explore.html">AutoML based on reinforcement learning</a> or <a href="https://en.wikipedia.org/wiki/Hyperparameter_optimization#Evolutionary_optimization">evolutionary algorithms</a> if you're curious.

**One approach to hyperparameter selection: simply search!**

# Searching for good hyperparameters

Two standard ways are 
1. **brute-force search**: try all parameter combinations within a specified range, and 
2. **random search**: try out random combinations of parameter setting within a specified range

A third way is to be more clever and use the results obtained from previous parameter settings to select a next setting that is expected to be better. This leads to things like 
3. **bayesian hyperparameter optimization** 

and also **evolutionary hyperparameter optimization** (not covered here).

One brute force search method often used is **grid search**. In cases where it makes sense to search through a very large space of parameter settings, or cases where each time you try a setting you have to do a lot of compute, it's better to use random search than grid search.

Let's get concrete and try these out on some model trained on some data.

# Data

To make the story clearer, we'll look at a relatively simple data set, the California housing data, and only tune a few hyperparameters.

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(data_home='./part4data')
df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
y = housing.target

In [3]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


Split off some test data:

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, random_state=42)

We'll measure performance using mean squared error:

In [5]:
from sklearn.metrics import mean_squared_error

# The base model

We first create a model using all the standard hyperparameters:

In [6]:
from sklearn.ensemble import RandomForestRegressor

In [7]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

We can check what the standard settings are:

In [8]:
rf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

Train:

In [9]:
rf.fit(X_train, y_train)

/home/alex/anaconda3/envs/dat158/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

Predict on test:

In [10]:
y_pred = rf.predict(X_test)

Let's check how the model did:

In [11]:
mean_squared_error(y_test, y_pred)

0.27691073678972666

Now, let's try to improve our baseline model by tweaking hyperparameters:

# Hyperparameter optimization

## Grid search

In grid search we select some paramaters to change, make a set or range of values for each of them, and try every combination. 

We specify our grid as a Python dictionary, or a list of dictionaries if we want to be more specific about parameter combinations to try (for example, "if `n_estimators` is 10, try `max_depth` 2, 3 and 4")

Here's a basic grid:

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [10, 100, 150, 200],
    'max_depth': [5, 50, 100, 150, None]
}

We can search over all the $4*5=120$ different settings, using 3-fold cross validation to check the performance of each one (training the model a total of $20*3 = 60$ times):

In [13]:
gs_reg = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=1)

In [14]:
gs_reg.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   35.0s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=10, n_jobs=-1,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [5, 50, 100, 150, None],
 

We grab the best model

In [15]:
best_reg = gs_reg.best_estimator_

...and check its parameters:

In [16]:
best_reg

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [17]:
gs_reg.best_params_

{'max_depth': 50, 'n_estimators': 200}

Let's try it on our test data:

In [18]:
y_pred_best = best_reg.predict(X_test)
mean_squared_error(y_test, y_pred_best)

0.25259951944145287

An improvement over the baseline.

## Random search

Let's try randomly searching through the space of parameters a specified number of iterations. This can be achieved using `RandomizedSearchCV`.

We can use scikit-learn's `RandomizedSearchCV` in a similar way to `GridSearchCV`: we specify the estimator to use and the parameter grid to search through. But we also specify the number of settings to try, and the method searches randomly through the parameter space that many times.

In [19]:
from sklearn.model_selection import RandomizedSearchCV

In [20]:
#?RandomizedSearchCV

In [21]:
n_iter = 5

In [22]:
rs_reg = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=n_iter, cv=3, verbose=1, random_state=42)

In [23]:
rs_reg.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    6.9s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=10, n_jobs=-1,
                                                   oob_score=False,
                                                   random_state=42, ver

Since we're only searching through a selection of the settings, this will of course not outperform the brute-force grid search approach.

In [24]:
mean_squared_error(y_test,rs_reg.predict(X_test))

0.25259951944145287

As our data is small it doesn't cost much time to search through *all* the settings in our above parameter grid. That is, using grid search is okay. However, if the parameter space was chosen to be much larger, our data set was more complicated, or our model took much longer to train, random search would be a better approach

## More advanced techniques: Bayesian optimization

Instead of searching through parameter settings one at a time, without paying attention to what's happened in the past, we can use methods for more intelligent selection of settings to try next. This can significantly reduce the number of iterations necessary to find good parameter settings, and can be useful when looking through a large number of options.

There are several libraries that can achieve this, one of which is `hyperopt`: https://github.com/hyperopt/hyperopt. 

> Using ML-techniques to automate the construction of machine learning systems is called **AutoML**, a field of research that has gotten significant attention in recent years (there are also multiple products offering AutoML solutions, e.g. Google's [Cloud AutoML](https://cloud.google.com/automl/)).

### `hyperopt`

In [26]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

We define a function that we want to minimize during the optimization process. In our case we want to minimize the mean absolute error of our random forest model, as measured using cross validation:

In [27]:
from sklearn.model_selection import cross_val_score

def objective(params):
    print(f"Using paramaters {params}")
    # Our model
    model = RandomForestRegressor(**params, n_jobs=-1, random_state=42)
    # The max cross-val score with current paramaters:
    score = cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()
    # We want to minimize the mean absolute error loss, not its negative:
    loss = -score
    return {'loss': loss, 'status': STATUS_OK}

Then we select a space of paramaters over which to search. We set it up similar to the above `param_grid`, using hyperopt to choose random parameter settings:

In [30]:
param_space = {
    'n_estimators': hp.choice('n_estimators', [10, 100, 150, 200]),
    'max_depth': hp.choice('max_depth', [5, 50, 100, 150, None])
}

We can manually select a random (stochastic) samples from this space, just to check that it works:

In [31]:
import hyperopt.pyll.stochastic

In [32]:
print(hyperopt.pyll.stochastic.sample(param_space))

{'max_depth': 5, 'n_estimators': 200}


Let's minimize our objective. 

In [33]:
# By using the trials object we can check what goes on in each trial:
trials = Trials()

In [34]:
best = fmin(fn=objective, space=param_space, algo=tpe.suggest, max_evals=5, trials=trials)

Using paramaters {'max_depth': 50, 'n_estimators': 10}
Using paramaters {'max_depth': None, 'n_estimators': 10}                     
Using paramaters {'max_depth': 100, 'n_estimators': 150}                     
Using paramaters {'max_depth': 100, 'n_estimators': 10}                      
Using paramaters {'max_depth': 100, 'n_estimators': 100}                     
100%|██████████| 5/5 [00:05<00:00,  1.13s/it, best loss: 0.27030255510507395]


The indices for the best parameters are:

In [35]:
best

{'max_depth': 2, 'n_estimators': 2}

We can use the `space_eval` function to find the corresponding values:

In [36]:
space_eval(param_space, best)

{'max_depth': 100, 'n_estimators': 150}

...and train and evaluate a model using those values:

In [37]:
model = RandomForestRegressor(**space_eval(param_space, best), random_state=42, n_jobs=-1)

In [38]:
model.fit(X_train, y_train)
mean_squared_error(y_test,model.predict(X_test))

0.25302251885744065

# MORE CONTENT TO BE ADDED